In [ ]:
# !pip install keras_preprocessing
# !pip install scikeras
# !pip install scikit-elm
# !pip install scikit-learn
# !python -m pip install --upgrade pip

In [ ]:
# --- Logging ---
import logging

# --- Operating System and File Handling ---
import os
import glob
import pickle

# --- Data Handling ---
import pandas as pd
import numpy as np

# --- Image Processing ---
import cv2
from skimage import io

# --- TensorFlow and Keras ---
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (
    Dense, Activation, Flatten, Dropout, BatchNormalization,
    GlobalAveragePooling2D, Conv2D, MaxPooling2D,
    GlobalAveragePooling3D, Conv3D, MaxPooling3D
)
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.callbacks import TensorBoard
from keras import backend as K

# --- Scikit-learn ---
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# --- Other Utilities ---
import matplotlib.pyplot as plt
import warnings
import tqdm
import time
import psutil

# --- Suppress Warnings ---
warnings.simplefilter('ignore')

In [ ]:
np.random.seed(0)

### Config

In [ ]:
class config:
    IMG_SIZE = 128
    BATCH_SIZE = 32
    EPOCHS = 100
    N_SPLITS = 5

In [ ]:

class Conv3DCNNTrainer:
    def __init__(self, data_path, labels_path, methods, epochs=50, batch_size=8, n_splits=5):
        self.data_path = data_path
        self.labels_path = labels_path
        self.methods = methods
        self.epochs = epochs
        self.batch_size = batch_size
        self.n_splits = n_splits
        self.data, self.labels = self.load_data()
        
    def setup_logging(self, method):
        log_file = f"/kaggle/working/{method}_training_log.log"
        
        # Ensure the directory exists
        os.makedirs(os.path.dirname(log_file), exist_ok=True)
        
        # Create a new logger instance
        logger = logging.getLogger(method)
        logger.setLevel(logging.INFO)
        
        # Avoid duplicate handlers
        if not logger.hasHandlers():
            # File handler (logs to file)
            file_handler = logging.FileHandler(log_file)
            file_handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
            
            # Stream handler (prints to console)
            stream_handler = logging.StreamHandler()
            stream_handler.setFormatter(logging.Formatter('%(message)s'))  # Cleaner console output
            
            # Add handlers to logger
            logger.addHandler(file_handler)
            logger.addHandler(stream_handler)
        
        return logger


    def load_data(self):
        with open(self.data_path, "rb") as f:
            data = pickle.load(f)
        with open(self.labels_path, "rb") as f:
            labels = pickle.load(f)
        return data, labels

    def define_classes(self, method):
        class_map = {
            "combined": ['Arguing','Eating_in_classroom','Explaining_the_Subject','HandRaise', 'Holding_Book','Holding_Mobile_Phone','Reading_Book','Sitting_on_Desk', 'Writing_On_Board','Writting_on_Textbook'],
            "student": ['Arguing','Eating_in_classroom','HandRaise','Reading_Book', 'Sitting_on_Desk','Writting_on_Textbook'],
            "teacher": ['Explaining_the_Subject','Holding_Book','Holding_Mobile_Phone','Writing_On_Board']
        }
        return class_map.get(method, [])

    def build_model(self, input_shape, num_classes):
        model = Sequential([
            Conv3D(64, kernel_size=3, activation="relu", padding="same", input_shape=input_shape),
            MaxPooling3D(pool_size=2),
            BatchNormalization(),
            
            Conv3D(64, kernel_size=3, activation="relu", padding="same"),
            MaxPooling3D(pool_size=2),
            BatchNormalization(),
            
            Conv3D(128, kernel_size=3, activation="relu", padding="same"),
            MaxPooling3D(pool_size=2),
            BatchNormalization(),
            
            GlobalAveragePooling3D(),
            Dense(512, activation="relu"),
            Dropout(0.3),
            Dense(num_classes, activation="softmax")
        ])
        
        lr_schedule = ExponentialDecay(
            initial_learning_rate=0.0001, decay_steps=100000, decay_rate=0.96, staircase=True
        )
        
        model.compile(
            loss="categorical_crossentropy",
            optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
            metrics=["accuracy"]
        )
        return model

    def train_with_kfold(self, method):
        classes = self.define_classes(method)

        log_file = self.setup_logging(method)
        skf = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=42)
        fold_no = 1
        all_fold_times, all_metrics = [], []
        input_shape = (self.data.shape[1], 128, 128, 3)
        tensorboard_dir = f"/kaggle/working/tensorboard_logs/{method}"
        model_dir = f"/kaggle/working/saved_models/{method}"
        os.makedirs(tensorboard_dir, exist_ok=True)
        os.makedirs(model_dir, exist_ok=True)

        for train_idx, val_idx in skf.split(self.data, self.labels):
            start_time = time.time()
            start_memory = psutil.Process().memory_info().rss / (1024 * 1024)            

            x_train_df = self.data[train_idx]
            x_valid_df = self.data[val_idx]
            y_train_labels = self.labels[train_idx]
            y_valid_labels = self.labels[val_idx]

            
            x_valid, x_test, y_valid, y_test = train_test_split(x_valid_df, y_valid_labels, test_size=0.5, random_state=42)


            y_train = tf.keras.utils.to_categorical(y_train_labels, num_classes=len(classes))
            y_valid = tf.keras.utils.to_categorical(y_valid, num_classes=len(classes))
            y_test = tf.keras.utils.to_categorical(y_test, num_classes=len(classes))

            model = self.build_model(input_shape, len(classes))
            tensorboard_callback = TensorBoard(log_dir=f'{tensorboard_dir}/fold_{fold_no}')

            history = model.fit(
                x_train_df, y_train,
                validation_data=(x_valid, y_valid),
                epochs=self.epochs,
                batch_size=self.batch_size,
                callbacks=[tensorboard_callback],
                verbose=0
            )
            
            model.save(f"{model_dir}/fold_{fold_no}.h5")

            y_pred_test = np.argmax(model.predict(x_test), axis=1)
            y_true_test = np.argmax(y_test, axis=1)

            test_acc = accuracy_score(y_true_test, y_pred_test)
            test_precision = precision_score(y_true_test, y_pred_test, average='weighted')
            test_recall = recall_score(y_true_test, y_pred_test, average='weighted')
            test_f1 = f1_score(y_true_test, y_pred_test, average='weighted')
            test_conf_matrix = confusion_matrix(y_true_test, y_pred_test)

            train_acc = history.history['accuracy'][-1]
            val_acc = history.history['val_accuracy'][-1]
            
            fold_time = time.time() - start_time
            memory_usage = (psutil.Process().memory_info().rss / (1024 * 1024)) - start_memory
            
            logging.info(f"Fold {fold_no}: Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}")
            logging.info(f"Test Metrics - Acc: {test_acc:.4f}, Prec: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}")
            logging.info(f"Time: {fold_time:.2f}s, Memory: {memory_usage:.2f}MB")
            logging.info(f"Confusion Matrix (Test Set):\n{test_conf_matrix}")
        
            print(f"Fold {fold_no}: Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}")
            print(f"Test Metrics - Acc: {test_acc:.4f}, Prec: {test_precision:.4f}, Recall: {test_recall:.4f}, F1: {test_f1:.4f}")
            print(f"Time: {fold_time:.2f}s, Memory: {memory_usage:.2f}MB")
            print(f"Confusion Matrix (Test Set):\n{test_conf_matrix}")

            all_fold_times.append(fold_time)
            all_metrics.append((test_acc, test_precision, test_recall, test_f1))
            fold_no += 1
        
        avg_metrics = np.mean(all_metrics, axis=0)
        logging.info(f"Overall - Accuracy: {avg_metrics[0]:.4f}, Precision: {avg_metrics[1]:.4f}, Recall: {avg_metrics[2]:.4f}, F1 Score: {avg_metrics[3]:.4f}")
        logging.info(f"Total Time: {sum(all_fold_times):.2f}s, Avg Time per Fold: {np.mean(all_fold_times):.2f}s")

        print(f"Overall - Accuracy: {avg_metrics[0]:.4f}, Precision: {avg_metrics[1]:.4f}, Recall: {avg_metrics[2]:.4f}, F1 Score: {avg_metrics[3]:.4f}")
        print(f"Total Time: {sum(all_fold_times):.2f}s, Avg Time per Fold: {np.mean(all_fold_times):.2f}s")
            
    
    def run(self):
        for method in self.methods:
            self.train_with_kfold(method)

In [ ]:
method_paths = {
    "combined": {
        "data_path": "/kaggle/input/threedpickles/classroom/classroom/data.pkl",
        "labels_path": "/kaggle/input/threedpickles/classroom/classroom/labels.pkl"
    },
    "student": {
        "data_path": "/kaggle/input/threedpickles/student/student/data.pkl",
        "labels_path": "/kaggle/input/threedpickles/student/student/labels.pkl"
    },
    "teacher": {
        "data_path": "/kaggle/input/threedpickles/teacher/teacher/data.pkl",
        "labels_path": "/kaggle/input/threedpickles/teacher/teacher/labels.pkl"
    }
}


for method in ["combined", "student", "teacher"]:
    trainer = Conv3DCNNTrainer(
        data_path=method_paths[method]["data_path"],
        labels_path=method_paths[method]["labels_path"],
        methods=[method],
        epochs=config.EPOCHS,
        batch_size=config.BATCH_SIZE,
        n_splits=config.N_SPLITS
    )

    print(f"Running training for method: {method}")
    trainer.run()
